<a href="https://colab.research.google.com/github/CryptoRobotFr/easy_backtest/blob/main/grid_trading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install python-binance

In [2]:
# Importation des librairies
import pandas as pd
from binance.client import Client
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

# Variables pour les données
client = Client()
pair_symbol = "BTCUSDT"
time_interval = Client.KLINE_INTERVAL_1HOUR
start_date = "01 january 2017"

# Récupération des données
klinesT = client.get_historical_klines(pair_symbol, time_interval, start_date)

# Créer un tableau grâce aux données
df = pd.DataFrame(klinesT, columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_av', 'trades', 'tb_base_av', 'tb_quote_av', 'ignore' ])

# Supprime les colonnes inutiles
df.drop(columns = df.columns.difference(['timestamp','open','high','low','close','volume']), inplace=True)

# Convertit les colonnes en numéric
for col in df.columns:
    df[col] = pd.to_numeric(df[col])

# Convertit les dates dans un format lisible
df = df.set_index(df['timestamp'])
df.index = pd.to_datetime(df.index, unit='ms')
del df['timestamp']

# Affiche le tableau
df

,open,high,low,close,volume
timestamp,,,,,
2017-08-17 04:00:00,4261.48,4313.62,4261.32,4308.83,47.181009
2017-08-17 05:00:00,4308.83,4328.69,4291.37,4315.32,23.234916
2017-08-17 06:00:00,4330.29,4345.45,4309.37,4324.35,7.229691
2017-08-17 07:00:00,4316.62,4349.99,4287.41,4349.99,4.443249
2017-08-17 08:00:00,4333.32,4377.85,4333.32,4360.69,0.972807
...,...,...,...,...,...
2022-04-24 12:00:00,39609.11,39660.00,39525.49,39582.37,802.989390
2022-04-24 13:00:00,39582.38,39636.12,39351.58,39579.70,1903.343860
2022-04-24 14:00:00,39579.70,39940.00,39371.46,39752.40,2319.757150


In [3]:
# --- Function definition ---
def custom_grid(first_price, last_order_down = 0.5, last_order_up = 1, down_grid_len=50, up_grid_len=50):
    """Function that create 2 array grid_buy and grid_sell

    Args:
        first_price (float): price at the begining
        last_order_down (float, optional): Percentage of the last grid buy. Defaults to 0.5.
        last_order_up (int, optional): Percentage of the last grid sell. Defaults to 1.
        down_grid_len (int, optional): Initial length of grid buy. Defaults to 50.
        up_grid_len (int, optional): Initial length of grid sell. Defaults to 50.

    Returns:
        array: return 2 array of float for price.
    """    
    down_pct_unity = last_order_down / down_grid_len
    up_pct_unity = last_order_up / up_grid_len

    grid_sell = []
    grid_buy = []

    for i in range(down_grid_len):
        grid_buy.append(first_price - first_price*down_pct_unity*(i+1))

    for i in range(up_grid_len):
        grid_sell.append(first_price + first_price*up_pct_unity*(i+1))

    return grid_buy, grid_sell

In [7]:
dt = df.copy().loc[:] # Initial test
first_price = dt.iloc[0]['close']

last_order_down = 0.35
last_order_up = 16

down_grid_len = 30
up_grid_len = 70

grid_buy, grid_sell = custom_grid(first_price, last_order_down = last_order_down, last_order_up = last_order_up, down_grid_len=down_grid_len, up_grid_len=up_grid_len) # Initial nice test
grid_buy

[4258.560316666667,
 4208.290633333333,
 4158.02095,
 4107.751266666666,
 4057.481583333333,
 4007.2119,
 3956.942216666667,
 3906.6725333333334,
 3856.40285,
 3806.1331666666665,
 3755.8634833333335,
 3705.5938,
 3655.3241166666667,
 3605.0544333333332,
 3554.78475,
 3504.515066666667,
 3454.2453833333334,
 3403.9757,
 3353.706016666667,
 3303.4363333333336,
 3253.16665,
 3202.8969666666667,
 3152.6272833333333,
 3102.3576000000003,
 3052.087916666667,
 3001.8182333333334,
 2951.5485500000004,
 2901.2788666666665,
 2851.0091833333336,
 2800.7395]

In [6]:
dt = df.copy().loc[:] # Initial test



last_order_down = 0.35
last_order_up = 16

down_grid_len = 30
up_grid_len = 70

grid_buy, grid_sell = custom_grid(first_price, last_order_down = last_order_down, last_order_up = last_order_up, down_grid_len=down_grid_len, up_grid_len=up_grid_len) # Initial nice test

trade_list = []
grid_buy_to_insert = 0
grid_sell_to_insert = 0

usd = 500
crypto = 500 / first_price

print("Starting price", first_price)
nb_same_index = 0
for index, row in dt.iterrows():

    try:
        if grid_buy_to_insert > 0:
            # print(row)
            grid_buy_diff = (row["open"] - grid_buy[0]) / (grid_buy_to_insert + 1)
            for i in range(grid_buy_to_insert):
                # print("grid buy", grid_buy[0]+grid_buy_diff)
                grid_buy.insert(0, grid_buy[0]+grid_buy_diff)
        
        if grid_sell_to_insert > 0:
            # print(row)
            grid_sell_diff = (grid_sell[0] - row["open"]) / (grid_sell_to_insert + 1)
            for i in range(grid_sell_to_insert):
                # print("grid_sell", grid_sell[0]-grid_sell_diff)
                grid_sell.insert(0, grid_sell[0]-grid_sell_diff)
        
    except:
        pass

    grid_buy_to_insert = 0
    grid_sell_to_insert = 0

    # if len(grid_buy) + len(grid_sell) != 120:
    #     print("error grid")

    if len(grid_buy) == 0 and usd < 0.05 * (crypto * row["open"]):
        print("End of buy grid => reset wallet and grid", index)
        # --- You can change perameters here for end of grid buy ---
        grid_buy, grid_sell = custom_grid(row["open"], last_order_down = 0.3, last_order_up = 1, down_grid_len=40, up_grid_len=60)
        usd = 0.5 * (usd + crypto * row["open"])
        crypto = 0.5 * (usd + crypto * row["open"]) /  row["open"]

    elif len(grid_sell) == 0 and (crypto * row["open"]) < 0.05 * usd:
        print("End of sell grid => reset wallet and grid", index)
        # --- You can change perameters here for end of grid sell ---
        grid_buy, grid_sell = custom_grid(row["open"], last_order_down = 0.3, last_order_up = 1, down_grid_len=40, up_grid_len=60)
        usd = 0.5 * (usd + crypto * row["open"])
        crypto = 0.5 * (usd + crypto * row["open"]) /  row["open"]

    check_same_index = False
    # -- BUY --
    
    if row["high"] > grid_sell[0]:
        try:
            while row["high"] > grid_sell[0]:
                crypt_to_sell = crypto / len(grid_sell)
                crypto -= crypt_to_sell
                usd += (crypt_to_sell * grid_sell[0])
                trade_list.append({
                    "date": index,
                    "side": "Sell",
                    "usd_amount": crypt_to_sell * grid_sell[0],
                    "price": grid_sell[0],
                    "usd": usd,
                    "crypto": crypto,
                    "wallet": usd + crypto*grid_sell[0]
                })
                grid_buy_to_insert += 1
                del grid_sell[0] 
                check_same_index = True
        except:
            print("End of grid sell",row["close"], index)
            pass

    # -- BUY --
    
    if row["low"] < grid_buy[0]:
        try:
            if check_same_index == True:
                nb_same_index += 1
                # print((row["high"]-row["low"])/row["close"])
            while row["low"] < grid_buy[0]:
                buy_usd_amount = usd/len(grid_buy)
                crypto += (buy_usd_amount / grid_buy[0])
                usd -= buy_usd_amount
                trade_list.append({
                    "date": index,
                    "side": "Buy",
                    "usd_amount": buy_usd_amount,
                    "price": grid_buy[0],
                    "usd": usd,
                    "crypto": crypto,
                    "wallet": usd + crypto*grid_buy[0]
                })
                grid_sell_to_insert += 1
                del grid_buy[0]
        except:
            print("End of grid buy",row["close"], index)
            pass


    
print("Number of same index", nb_same_index)

Starting price 4308.83
Number of same index 34


In [5]:
df_trades = pd.DataFrame(trade_list).iloc[:]
df_trades['wallet_ath'] = df_trades['wallet'].cummax()
df_trades['price_ath'] = df_trades['price'].cummax()
df_trades['wallet_drawdown_pct'] = (df_trades['wallet_ath'] - df_trades['wallet']) / df_trades['wallet_ath']
df_trades['price_drawdown_pct'] = (df_trades['price_ath'] - df_trades['price']) / df_trades['price_ath']
max_trades_drawdown = df_trades['wallet_drawdown_pct'].max()
max_price_drawdown = df_trades['price_drawdown_pct'].max()
wallet_perf = (df_trades.iloc[-1]['wallet'] - df_trades.iloc[0]['wallet']) / df_trades.iloc[0]['wallet']
price_perf = (df_trades.iloc[-1]['price'] - df_trades.iloc[0]['price']) / df_trades.iloc[0]['price']
print("Total trades:", len(df_trades))
print("\n--- Wallet ---")
print("Wallet performance: {}%".format(round(wallet_perf*100, 2)))
print("Worst Wallet Drawdown: -{}%".format(round(max_trades_drawdown*100, 2)))
print("\n--- Asset ---")
print("Asset performance: {}%".format(round(price_perf*100, 2)))
print("Worst Asset Drawdown: -{}%".format(round(max_price_drawdown*100, 2)))
df_trades

Total trades: 2946

--- Wallet ---
Wallet performance: 1061.99%
Worst Wallet Drawdown: -64.81%

--- Asset ---
Asset performance: 827.39%
Worst Asset Drawdown: -83.23%


,date,side,usd_amount,price,usd,crypto,wallet,wallet_ath,price_ath,wallet_drawdown_pct,price_drawdown_pct
0,2017-08-17 16:00:00,Buy,16.666667,4258.560317,483.333333,0.119954,994.166667,994.166667,4258.560317,0.000000,0.000000
1,2017-08-17 19:00:00,Buy,16.666667,4208.290633,466.666667,0.123915,988.136594,994.166667,4258.560317,0.006065,0.011804
2,2017-08-18 01:00:00,Buy,16.666667,4158.020950,450.000000,0.127923,981.907431,994.166667,4258.560317,0.012331,0.023609
3,2017-08-18 17:00:00,Buy,16.666667,4107.751267,433.333333,0.131981,975.476771,994.166667,4258.560317,0.018800,0.035413
4,2017-08-18 18:00:00,Sell,7.651326,4290.010339,440.984659,0.130197,999.531431,999.531431,4290.010339,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
2941,2022-04-14 18:00:00,Buy,117.536613,39749.520343,7522.343254,0.101830,11570.022945,12415.795323,68581.584143,0.068121,0.420405
2942,2022-04-18 18:00:00,Sell,115.918994,40981.028927,7638.262248,0.099001,11695.427031,12415.795323,68581.584143,0.058020,0.402448
2943,2022-04-21 10:00:00,Sell,119.658515,42303.067855,7757.920763,0.096172,11826.310269,12415.795323,68581.584143,0.047479,0.383172
2944,2022-04-21 20:00:00,Buy,117.544254,40873.782671,7640.376509,0.099048,11688.852417,12415.795323,68581.584143,0.058550,0.404012
